#### 高雄捷運每日各站分時資料
- 測試資料
- 資料時間: 2024.05.31

In [1]:
import pandas as pd

In [3]:
file_path_O = "./data/00354_sample.csv"
# file_path_D = "./data/00355_sample.csv"

data_O = pd.read_csv(file_path_O)
# data_D = pd.read_csv(file_path_D)

In [3]:
# 調整資料
# 刪除第一個 row 並將第二個 row 當成 header
data_O.columns = data_O.iloc[0]  # 將第0個 row 當成 header
df_O = data_O[1: ]  # 刪除第一個row（原來的 header 和第一個 row）

# 重設index
df_O.reset_index(drop = True, inplace = True)
df_O.head(10)

,TripDate,TripHour,TicketClass,IDType,HolderType,OriginStationID,OriginStationName,DestinationStationID,DestinationStationName,Volume,InfoDate
0,2024-05-31,15,IC,iPASS,A,R20,後勁,R4,高雄國際機場,1,2024-05-31
1,2024-05-31,23,IC,iPASS,C01,R11,高雄車站,R8,三多商圈,1,2024-05-31
2,2024-05-31,15,IC,iPASS,C02,O13,大東,O6,信義國小,1,2024-05-31
3,2024-05-31,19,IC,iPASS,C01,R8,三多商圈,O2,鹽埕埔,2,2024-05-31
4,2024-05-31,20,IC,iPASS,A,R22,青埔,R14,巨蛋,2,2024-05-31
5,2024-05-31,13,IC,iPASS,A,R4A,草衙,R9,中央公園,1,2024-05-31
6,2024-05-31,9,IC,iPASS,A,O6,信義國小,O13,大東,1,2024-05-31
7,2024-05-31,17,IC,iPASS,A,R19,楠梓科技園區,O7,文化中心,6,2024-05-31
8,2024-05-31,17,IC,iPASS,A,O13,大東,O4,前金,1,2024-05-31
9,2024-05-31,18,IC,iPASS,A,O7,文化中心,O5/R10,美麗島,8,2024-05-31


In [4]:
# 清理資料欄位
df_O.drop(["OriginStationName",
          "DestinationStationName", "InfoDate"], axis = 1, inplace = True)

C:\Users\JUN-MING-WU\AppData\Local\Temp\ipykernel_25856\1662197484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_O.drop(["OriginStationName",


In [5]:
# 調整時間欄位
# df_O
df_O.loc[:, 'TripHour'] = df_O['TripHour'].astype(int)
df_O.loc[:, 'adjusted_time'] = df_O['TripHour'].apply(lambda x: x if x >= 5 else x + 24)
df_O = df_O.sort_values(by = 'adjusted_time')

# 清理空值
df_O = df_O.dropna(subset = ["HolderType"])
print(df_O.shape)

(9526, 9)


C:\Users\JUN-MING-WU\AppData\Local\Temp\ipykernel_25856\3877785685.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_O.loc[:, 'adjusted_time'] = df_O['TripHour'].apply(lambda x: x if x >= 5 else x + 24)


In [6]:
def clean_data(df_O):
    # Change column type to object for column: 'Volume'
    df_O = df_O.astype({'Volume': 'int'})
    return df_O

df_O_clean = clean_data(df_O.copy())
df_O_clean.head()

,TripDate,TripHour,TicketClass,IDType,HolderType,OriginStationID,DestinationStationID,Volume,adjusted_time
636,2024-05-31,5,IC,iPASS,A,R16,R6,1,5
457,2024-05-31,5,IC,iPASS,A,O9,R16,2,5
885,2024-05-31,5,IC,iPASS,A,R3,R7,1,5
8835,2024-05-31,5,IC,iPASS,C02,OT1,O11,1,5
2773,2024-05-31,5,IC,iPASS,A,R4A,R20,1,5


#### 資料欄位描述
- IDType：識別碼類別
    - EasyCard-  電子票證 -悠遊卡 
    - iPASS-  電子票證 - 一卡通 
    - icash-  電子票證 - 愛金卡
    - HappyCash-  電子票證 - 有錢卡 
    - CreditCard-  信用卡
    - PaymentID-  電子支付
    
- HolderType：身分別
    - A-普通身分(含TPASS)； 
    - B-學生身分； 
    - C-優待身分
        - C01-敬老優待
        - C02-愛心優待
        - C09-其他優待


In [7]:
# 現在要先抓出跟MeNGo(Tpass)有關的資料(Type A)
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_O_clean):
    # Filter rows based on column: 'HolderType'
    df_O_clean = df_O_clean[df_O_clean['HolderType'] == "A"]
    # Sort by column: 'Volume' (descending)
    # 從最多流量的降冪排列，先看看狀況
    df_O_clean = df_O_clean.sort_values(['Volume'], ascending = [False])
    return df_O_clean

df_O_clean_1 = clean_data(df_O_clean.copy())
df_O_clean_1.head()

,TripDate,TripHour,TicketClass,IDType,HolderType,OriginStationID,DestinationStationID,Volume,adjusted_time
6956,2024-05-31,7,IC,iPASS,A,R16,R19,102,7
2812,2024-05-31,7,IC,iPASS,A,R3,R5,87,7
757,2024-05-31,7,IC,iPASS,A,R5,R3,75,7
6797,2024-05-31,7,IC,iPASS,A,R14,R19,67,7
1886,2024-05-31,6,IC,iPASS,A,R13,R3,67,6
